<img src="https://images.unsplash.com/photo-1630342654882-aed9e2e86f24?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=774&q=80" width="1000px" >

# RETAIL ANALYIS 

1. ## Data Pre-processing
2. ## RFM Analysis
3. ## Market Segmentation
4. ## Market Basket Analysis
4. ## Conclusion

In [200]:
#Imports
#Data cleaning 
import pandas as pd
from ast import literal_eval
from datetime import datetime as dt
from numpy import where

#plotting
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import seaborn as sns
sns.set(style='white')
import plotly.graph_objects as go

In [201]:
#READ DATA
#users= read_csv('../input/awssampleretaildemostore/users.csv')
users = pd.read_csv('../input/awssampleretaildemostore/raw_users.csv',
                    usecols =[0,1,2,3,4,5,6,7])
#items = read_csv('../input/awssampleretaildemostore/items.csv')
products = pd.read_csv('../input/awssampleretaildemostore/raw_products.csv', 
                       usecols = [0,3,4,5,7,10])
df_lst = pd.read_csv('../input/awssampleretaildemostore/interactions.csv',
                        iterator=True, chunksize=20000)

df = pd.concat(df_lst)    

## Data Cleaning and Exploration

In [202]:

users.columns = [c.lower() for c in users.columns]

# Parse address information
add_lst = ['address1', 'country', 'city', 'state', 'zipcode']
users.addresses = users.addresses.apply(lambda x: [(literal_eval(x[1:-1])).\
                                                   get(i) for i in add_lst])
users[add_lst] = pd.DataFrame(users.addresses.tolist())
users.drop('addresses', axis=1, inplace = True)
users.head()

In [203]:
# Clean Interaction data

#Convert timestamp
df.columns = [c.lower() for c in df.columns]
df['date'] = pd.to_datetime(df['timestamp'],  unit='s')
df.drop('timestamp', axis = 1, inplace = True)

#Check for duplicates
df.drop_duplicates(inplace = True)


In [204]:
products.rename(columns={'id': 'item_id', 'name': 'product_name'}, 
               inplace = True)

#Merge all datasets
df = df.merge(products, how='left').merge(
                                    users, how='left', left_on = 'user_id',
                                    right_on = 'id')
df.drop('id', axis = 1, inplace = True) 

### Missing data 
After the merge all fields have no missing data, in the *price* field this is intuitively not correct as some *event_type* should not have a price i.e *productViewed*. Thus set *order_completed* *event_type* to have price.

In [205]:
missing_value = df.isnull().sum()
#No missing values

In [206]:
#Remove price from non purchasing event_types
browse = ['ProductViewed', 'ProductAdded', 'CartViewed', 'CheckoutStarted']
df['price'] = where(df['event_type'].isin(browse), "", df['price'])
df['price'] = pd.to_numeric(df['price'])

## Products Categories Analysis
&emsp;  **Orders, Revenue, By Categories** <br>
1.  Which are the largest product categories by order numbers, revenue, and number of products sold?<br>
2.  How do the categories compare in terms of interactions and also conversion rates?<br>
3. Which are the top and bottom 10 performing products in terms of revenue?<br>
4. What is the average revenue per order by product categories? <br>

In [207]:
#How many Categories 
df_categories = df[~df['price'].isna()].groupby('category').agg(
                                       no_of_customers = ('user_id','nunique'),
                                       no_of_products = ('product_name', 'nunique'),
                                       revenue_by_category=('price', 'sum'),
                                       no_orders = ('price', 'count')
                                      ).reset_index()
df_categories['average_revenue_per_order'] = (df_categories['revenue_by_category']/
                                              df_categories['no_orders']).round(2)
df_categories

In [208]:
print(f'There are {df_categories.no_of_customers.sum()} unique customers.')
print(f'There are {df_categories.no_of_products.sum()} unique products sold.')
print(f'A total of {"${:,.2f}".format(df_categories.revenue_by_category.sum())} revenue was generated.')
print(f'{df_categories.no_orders.sum()} orders were fulfilled.')


In [209]:
figure, axis = plt.subplots(2, 2,figsize=(15, 15), facecolor='w', edgecolor='k')

a = 2 #rows
b = 2 # columns
c = 1 # subplot iterator

for i in df_categories.columns[1:-1]:
    plt.subplot(a, b, c)
    plt.title(f'{i}')
    plt.xlabel(i)
    plt.suptitle('Products Categories')
    plt.ylabel('')
    cmap = sns.diverging_palette(220, 20, as_cmap=True)
    sns.barplot(x = i, y = 'category', data = df_categories,
                order= df_categories.sort_values(
                    by=[i], ascending=False).set_index('category').index,
               palette = 'plasma')
    sns.despine(bottom = True, left = True)
    c = c + 1

plt.show()

In [210]:
#High ticket categories

plt.figure(figsize=(8,8))
sns.barplot(x = i, y = 'category', data = df_categories,
                order= df_categories.sort_values(
                    by=[i], ascending=False).set_index('category').index,
               palette = 'plasma')
plt.title('Average Revenue per order by Category')

sns.despine(bottom = True, left = True)

*Furniture*, *apparels*, *instruments* and *electronic* are the top categories by total revenue. Likely correlation between number of products per category and number of orders completed.
In addition *apparel*, *furniture*, *accessories* and *seasonal* categories have highest average revenue per order.

&emsp; **interactions by category**
1. What is the overall conversion rate for the store? <br>
2. What are the conversion raates for each product categories? <br>
3. Analyze customer flow. <br>

In [211]:
df_interactions = df[['user_id', 'event_type', 'category']]
df_interactions =(df_interactions.pivot_table(index=['category'],
                                    values = 'user_id',
                                    columns='event_type', aggfunc= 'count')
                                    .reset_index().rename_axis("", axis=1)
                                     )

inter_seq = ['category','ProductViewed','ProductAdded', 'CartViewed', 'CheckoutStarted',
             'OrderCompleted']

interactions = df_interactions[inter_seq].copy()
interactions['TotalInteractions'] = (interactions[interactions.columns[1:]].apply(
                                                lambda x: sum(x), axis = 1))
interactions.sort_values(by ='TotalInteractions', ascending=False, inplace=True)
interactions

In [212]:
#Calculate conversion rate
interactions['Conversion_rate'] = (interactions['OrderCompleted'] *100/
                                      interactions['TotalInteractions']).round(3)
interactions

In [213]:
print(f'The overall conversion rate is:\
       {"{:.3f}%".format(interactions.Conversion_rate.mean())}')

All product categories have a conversion rate in range 1.0 - 1.7% with *books* and *housewares* having the highest and lowest respectively.

In [214]:
df_interactions = df_interactions.set_index('category')


In [215]:
categories =df_interactions.index.values.tolist()
category_plot_names = []
fig = go.Figure()

buttons=[]

default_category = "apparel"

for cat in categories:
    x = sorted(df_interactions.loc[cat].values.tolist(), reverse=True)
    #print(x)
    fig.add_traces(go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = ['ProductViewed', "ProductAdded", 'CartViewed','Checkout', 'Sale' ],
      color = ["blue", "red", "yellow", "green", "purple", "cyan", "black"] 
    ),
    link = dict(
      source = [0, 1, 2, 3, 4],
      target = [1, 2, 3, 4],
      value = x
    ), visible=(cat==default_category)))
    
    category_plot_names.append(cat)
    
for cat in categories:
    buttons.append(dict(method='update',
                        label=cat,
                        args = [{'visible': [cat==r for r in category_plot_names]}])) 
    
# Add dropdown menus to the figure
fig.update_layout(showlegend=False, updatemenus=[{"buttons": buttons, "direction": "down", "active": categories.index(default_category), "showactive": True, "x": 0.5, "y": 1.15}])
fig.show()

## Time

In [216]:
df['date'].dt.month.unique()
#This is 3 Months of data june/july/ August

df['month'] = df['date'].dt.month_name()
              

In [217]:
df_t = df.groupby('month').agg(no_of_customers = ('user_id','nunique'),
                           no_of_products = ('product_name', 'nunique'),
                           revenue_by_category=('price', 'sum'),
                           no_orders = ('price', 'count')
                          ).reset_index()
#Reorder by Month
sort_order= ['June', 'July', 'August'] #category order
df_t.index = pd.CategoricalIndex(df_t['month'], categories=sort_order, ordered=True)
df_t = df_t.sort_index().reset_index(drop = True)   
df_t                        

In [218]:
#Weekly
df['day_of_year'] = df['date'].dt.strftime('%j')
df['day_of_week'] = df['date'].dt.strftime('%a')
df